Importing dependencies and loading csv file

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
ds=pd.read_csv("/content/drive/MyDrive/ML Projects/Twitter_live_Sentiment_Analysis/sentiment140.csv", encoding = "ISO-8859-1")

In [6]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id"     1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [7]:
ds.drop(columns=['id"','date','flag','user'],inplace=True)


In [8]:
y=ds['target']

In [9]:
ds.head()


,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [10]:
ds.duplicated().sum() #checking for duplicates

16309

In [11]:
ds=ds.drop_duplicates(keep='first')

Target value for all tweets is 0 or 4 at which implies negative and positive

In [12]:
ds = ds.sample(frac = 1)#shuffling

In [13]:
ds.head()

,target,text
1127270,4,http://bit.ly/15yUjx &lt; new 18 second video...
967774,4,@mellowdi I would if you were local but gon en...
682605,0,@dainiesaur &quot;She's strong enough to walk ...
116343,0,Spent the whole day in bed...not feeling so go...
1274992,4,@vckrs you. By a long shot although rob did f...


In [14]:
ds_split=np.array_split(ds,40)#splitting 1.6 Million tweets into 40 equal parts

In [15]:
ds_split[0]['target'].value_counts()

4    19818
0    19775
Name: target, dtype: int64

In [16]:
df=ds_split[0] #df stores the fraction of our original dataframe
df.head()

,target,text
1127270,4,http://bit.ly/15yUjx &lt; new 18 second video...
967774,4,@mellowdi I would if you were local but gon en...
682605,0,@dainiesaur &quot;She's strong enough to walk ...
116343,0,Spent the whole day in bed...not feeling so go...
1274992,4,@vckrs you. By a long shot although rob did f...


In [17]:
df=df.reset_index()

In [18]:
df.drop(columns=['index'],inplace=True)

In [19]:
df.head()

,target,text
0,4,http://bit.ly/15yUjx &lt; new 18 second video...
1,4,@mellowdi I would if you were local but gon en...
2,0,@dainiesaur &quot;She's strong enough to walk ...
3,0,Spent the whole day in bed...not feeling so go...
4,4,@vckrs you. By a long shot although rob did f...


In [20]:
df['target'].value_counts()

4    19818
0    19775
Name: target, dtype: int64

Preprocessing

In [21]:
!pip install nltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import nltk

In [23]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

In [25]:
from nltk.stem.porter import PorterStemmer

In [26]:
import re
import string

In [27]:

def pp(text):

  text=re.sub(r'https?:\/\/.*[\r\n]*','', text)
  text=re.sub(r'#','',text)

  tokenizer = TweetTokenizer(preserve_case=False,strip_handles=True, reduce_len=True)
  text= tokenizer.tokenize(text)

  y=[]
  for i in text:
    if i.isalnum():
      y.append(i)

  text=y[:]
  y.clear()

  stop_words = set(stopwords.words('english'))
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for i in text:
    if i not in stop_words and i not in punc:
      y.append(i)

  text=y[:]
  y.clear()

  ps=PorterStemmer()
  for i in text:
    y.append(ps.stem(i))

  return " ".join(y)


In [28]:
pp(df['text'][1])

'would local gon enjoy'

In [29]:
df['text'][1]

'@mellowdi I would if you were local but gon enjoy for you. '

In [30]:

df['pp_text']=df[df.columns[1]].apply(pp)
df.loc[df['target'] == 4, 'target'] = 1

In [31]:
df.head()

,target,text,pp_text
0,1,http://bit.ly/15yUjx &lt; new 18 second video...,
1,1,@mellowdi I would if you were local but gon en...,would local gon enjoy
2,0,@dainiesaur &quot;She's strong enough to walk ...,strong enough walk away broken enough look bac...
3,0,Spent the whole day in bed...not feeling so go...,spent whole day bed feel good today
4,1,@vckrs you. By a long shot although rob did f...,long shot although rob fall stair wretch gut n...


Dropping empty rows

In [32]:
df.dropna(inplace=True)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39593 entries, 0 to 39592
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   target   39593 non-null  int64 
 1   text     39593 non-null  object
 2   pp_text  39593 non-null  object
dtypes: int64(1), object(2)
memory usage: 928.1+ KB


Model

In [34]:
y=df['target'].values

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf= TfidfVectorizer(max_features=3750)
X=tfidf.fit_transform(df['pp_text']).toarray()
len(tfidf.vocabulary_)

3750

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size =0.3,
                                                    random_state=1)


In [37]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

models = {
    'Multinomial Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier()
}

results_df = pd.DataFrame(columns=['Accuracy', 'F1 Score', 'Precision', 'Recall'])

for model_name, model in models.items():

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    #evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    results_df.loc[model_name] = [accuracy, f1, precision, recall]


In [38]:
print(results_df)

                         Accuracy  F1 Score  Precision    Recall
Multinomial Naive Bayes  0.722091  0.720041   0.740967  0.700264
Logistic Regression      0.739434  0.748762   0.737094  0.760805
Random Forest            0.729500  0.737907   0.729869  0.746123
SVM                      0.739687  0.749189   0.736993  0.761795
K-Nearest Neighbors      0.564657  0.683169   0.543425  0.919663
Decision Tree            0.667789  0.676398   0.672538  0.680304


In [39]:
from sklearn.linear_model import LogisticRegression

lr= LogisticRegression()
lr=lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)

Pickle Files

In [40]:
import pickle
pickle.dump(tfidf,open('sentiment_analysis_vectorizer.pkl','wb'))
pickle.dump(lr,open('sentiment_analysis_model.pkl','wb'))
